In [5]:
# this package needs to be loaded to open a file dialogue to open the data file
#from tkinter import filedialog
#from tkinter import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math
import os.path
import os
import sys
import plotly.offline as py
from scipy.interpolate import interp1d
import scipy.fftpack as ft
py.init_notebook_mode(connected=True)

## Import the data file

In [ ]:
def data_import (filename, N_headers):
    data = []
    
    f = open(filename, 'r')
    for i in np.arange(N_headers):
        f.readline()
    
    for line in f:
        line=line.strip()
        line=line.split()
        
        for i in np.arange(len(line)):
            line[i] = float(line[i])
        
        data.append(line)
    
    data = np.array(data)
        
    final_data = []
    for i in np.arange(len(line)):
        final_data.append(data[:,i])
    
    return np.array(final_data)

## Save the final data

In [ ]:
def save (data, names, units, comments, filename):
    Units = ''
    Comments = ''
    Names = ''
    
    for i in np.arange(len(data)-1):
        Units = Units + units[i] + '\t'
        Comments = Comments + comments[i] + '\t'
        Names = Names + names[i] + '\t'
           
    Units = Units + units[len(data)-1] + '\n'
    Comments = Comments + comments[len(data)-1] + '\n'
    Names = Names + names[len(data)-1] + '\n'
    
       
    if os.path.isfile(filename) == True:
        x='w'
    else:
        x='x'
        
    with open(filename, x) as g:
        g.write(Names)
        g.write(Units)
        g.write(Comments)
        for i in np.arange(len(data[0])):
            Data = ''
            
            for j in np.arange(len(data)-1):
                Data = Data + str(data[j][i]) + '\t'
            
            Data = Data + str(data[-1][i]) + '\n'
            
            g.write(Data)

## from the filename a string with the information about frequency and temperature is extracted

In [ ]:
def comment ( filename ) :
        
    namefile = filename[::-1]
    position = namefile.find('\\')
    file = filename[-position:]
    
    
    # temperature sweep
    if file[0] == '-':
        namefile = filename[::-1]
        position = namefile.find('\\')
        file = filename[-position:]
        
        fb = file.find('F')
        fe = file.find('G')
        Tb = file.find('T')
        Te = file.find('K')
        
        comment = file[Tb+1:Te]+'K '+file[fb+1:fe]+'GHz'
        
    
    # individual measurement
    else:
        F = file.find('F')
        p = file.find('p')
        T = file.find('T')
        K = file.find('K')
        
        if file[F+1] == '0':
            frequency = file[F+2:p]+'.'+file[p+1:T]+'GHz'
        
        else:
            frequency = file[F+1:p]+'.'+file[p+1:T]+'GHz'
            
        if file[T+1] == '0':
            Temp = file[T+2:K+1]
        else:
            Temp = file[T+1:K+1]
            
        comment = Temp+' '+frequency
    
    return (comment)

## Switch two data sets

In [2]:
def swap (data1, data2):
    return [data2, data1]

## Plot amplitude and phase
this function shows a plot of the amplitude and phase against the magnetic field in the same diagram;
amlitude and phase share the x-axis but have different y-axes

In [ ]:
def plot (Amp, Phase, Field):
    fig, ax1 = plt.subplots()
    
    ax1.set_xlabel('field')
    ax1.set_ylabel('amp', color = 'red')
    ax1.plot(Field, Amp, color = 'red')
    ax1.tick_params(axis='y', labelcolor = 'red')
    
    ax2 = ax1.twinx()
    
    ax2.set_ylabel('phase', color = 'navy')
    ax2.plot(Field, Phase, color = 'navy')
    ax2.tick_params(axis='y', labelcolor = 'navy')
    
    fig.tight_layout()
    plt.show()

## Remove jumps from a data set

In [ ]:
def jumps (data, Field, dev):
    # I am taking the first (dphase) and second (ddphase) derivative of the phase to see jumps more clearly
    # I will later use that continuous signals usually flatten with higher derivatives whereas jumps don'taking
    # another reason for using the second and not the first derivative is that at a jump the second derivative will
    # show two peaks (one positive and one negative) whereas there will be only one at the first derivative
    # this gives us another criteria to distinguish between jumps and just big but continuous derivatives
    
    # note that I am not really taking a derivative since I am not deviding by deltaB, but assuming that the field steps
    # are equidistant that will only result in an overall factor
    
    # first derivative
    # note that I am taking the derivative only from one side;
    # taking it from both would give me two positive peaks already in the first derivative
    # so for simplicity I am only taking a one-sided derivative
    ddata = []
    for i in np.arange(len(data)):
        if i < len(data)-1:
            ddata.append(data[i+1]-data[i])
        else:
            ddata.append(data[i]-data[i-1])
    
    ddata = np.array(ddata)
    
    # second derivative
    dddata = []
    for i in np.arange(len(data)):
        if i < len(data)-1:
            dddata.append(ddata[i+1]-ddata[i])
        else:
            dddata.append(ddata[i]-ddata[i-1])
    
    dddata = np.array(dddata)
    
    
    # in this step I am calculating the mean and standard deviation of the second derivative
    ave = np.mean(abs(dddata))
    std = np.std(abs(dddata))
    
    
    # here I am finding the jumps (i.e their indices in the respective arrays)
    # jumps are features that are "deviation" times the standard deviation away from the mean of the second derivative
    deviation = dev
    
    
    # note that jumps also need to have two features right next to each other
    # remember: 0th derivative: jump
    #  1st derivative: 1 peak
    #  2nd derivative: 2 peaks right next to each other with opposite sign
    
    # for now this condition is fulfilled (because deviation=3)
    # I am using this while loop to being able to repeat the following step several times if I am not happy with the outcome
    # what I can change in between the different tries is "deviation"
    while deviation != '':
        
        # index gives you an array with the indices of suspected jumps
        # if there is no jump suspected this array will be empty
        # in order to distinguish the case of no jump and several jumps the condition len(index)>0 will be posed 
        # several times in the following
        index = np.arange(len(data))[abs(dddata)>ave+float(deviation)*std]
        
        # this is the case of several jumps
        if len(index) > 0:
            # I am checking if there are pairs of jumps right next to each other
            mask = [False]
            for i in (np.arange(len(index)-1)+1):
                if index[i-1] == index[i]-1:
                    mask.append(True)
                else:
                    mask.append(False)
                    
            
            # index finally gives you the indices of the jumps	
            index1 = index[mask]
            
            # now I am plotting the phase and dots on top of it where jumps are suspected
            # jumpsx and y give the phase and field of the jump
            # note that the jump occurs from the index given in index1 to the following data point
            # for plotting a dot where the jump is expected using the average values is most convenient
            jumpsx = [(Field[i]+Field[i+1])/2 for i in index1]
            jumpsy = [(data[i]+data[i+1])/2 for i in index1]
            
            
            plt.plot(Field, data, zorder = 0)
            plt.scatter(jumpsx, jumpsy, color='r', zorder=1, s=8)
            plt.show()
            print ('Number of jumps = '+str(len(jumpsx)))
            
            
            # now based on the plot above I am asking you whether the dots actually mark jumps
            # your answer options are:
            # - consecutive letters of 'n' and 'y' (e.g. nnyyyyny) for each dot from left to right; those will be the jumps that are removed in the following
            # - enter a number; this number will be a new value for "deviation" and we will repeat the search for jumps with this different value 
            # - enter 'end': this will interrupt the python code - note that you can do this at any point you are asked for input
            # - hit enter and you will continue with the amplitude
            jumps = input('which of the marked points are actually jumps (enter y/n for jump/no jump)? Or do you maybe want to change the number of standard deviations the jump has to be away from the mean (enter a number)? ' )
            if jumps == '':
                deviation = ''
                index = []
            elif jumps[0] == 'n' or jumps[0] == 'y':
                deviation = ''
            elif jumps == 'end':
                sys.exit()
            else:
                deviation = float(jumps)
                
        # this is the case of no jump; you have two options here:
        # - hit enter to continue with the same process for the amplitude
        # - enter a number; this number will replace the old value for "deviation" and the process of finding jumps will be repeated
        else:
            deviation = input('No jump was found. Do you want to continue with the amplitude (press Enter) or change the numbers of standard deviations the jump has to be away from the mean (enter a number)? ')
            if deviation == 'end':
                sys.exit()
                
                
    # if there was no jump found and you chose to continue with the amplitude then len(index) will still be zero 
    # so the code inside the if-loop will not be executed
    # otherwise it will be >0
    
    if len(index)>0:
        
        # from the string jumps that says 'y'/'n' if there is/isn't a jump a mask (manual_mask) is created
        manualmask = []
        for i in jumps:
            if i == 'n':
                manualmask.append(False)
            elif i == 'y':
                manualmask.append(True)
                
        # index2 now really gives the indeces of the jumps
        index2 = np.array(index1)[manualmask]
        
        # this is where the jumps in the phase are actually eliminated
        data_new = data
        # only if there are any jumps left and you didn't tell it that all the suspected points are in fact not jumps
        if len(index2)>0:
            # for all the jumps
            for i in index2:
                # I am creating an array that is 0 before and 1 after the jump is happening
                a = np.zeros(len(data))
                for k in np.arange(len(data)):
                    if k > i:
                        a[k] = 1
                # with this array I can just add the jump to all the values after it
                data_new = data_new + (data[i]-data[i+1])*a
    else:
        data_new = data
    
    plt.plot(Field, data_new)
    
    
    return data_new


## This defines a polynomial 9th order and a routine that fits an n-th order polynomial (max 9th) to a data set

In [ ]:
# this defines a 9th order polynomial
def polynomial(x, a, b, c, d, e, f, g, h, i, j):
    return (a*x**9 + b*x**8 + c*x**7 + d*x**6 + e*x**5 + f*x**4 + g*x**3 + h*x**2 + i*x + j)

# this function fits an n-th order polynomial to the amplitude data
# input variable of the function are:
# - n: the order of the polynomial you want to fit to the data
# - xmin: an array of the minimum field values of the signals you want to take out of the fit
# - xmax: ... maximum ...
# - the amplitude
# - the field
def fitpolynomial(n, xmin, xmax, amp, field):
    mask = np.array([[any([(field<xmin[j])[i],(field>xmax[j])[i]]) for i in np.arange(len(field))] for j in np.arange(len(xmax))])
    mask = [all([mask[j,i] for j in np.arange(len(xmax))]) for i in np.arange(len(field))]
    fieldcut = field[mask]
    ampcut = amp[mask]

    if n == 9:
        def fct(x, a, b, c, d, e, f, g, h, i, j):
            return polynomial(x, a, b, c, d, e, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7], parameters[8], parameters[9])
           
    elif n==8:
        def fct(x, b, c, d, e, f, g, h, i, j):
            return polynomial(x, 0, b, c, d, e, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7], parameters[8])
           
    elif n==7:
        def fct(x, c, d, e, f, g, h, i, j):
            return polynomial(x, 0, 0, c, d, e, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7])
         
    elif n==6:
        def fct(x, d, e, f, g, h, i, j):
            return polynomial(x, 0, 0, 0, d, e, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5], parameters[6])
               
    elif n==5:
        def fct(x, e, f, g, h, i, j):
            return polynomial(x, 0, 0, 0, 0, e, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5])
               
    elif n==4:
        def fct(x, f, g, h, i, j):
            return polynomial(x, 0, 0, 0, 0, 0, f, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])
              
    elif n==3:
        def fct(x, g, h, i, j):
            return polynomial(x, 0, 0, 0, 0, 0, 0, g, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2], parameters[3])
        
    elif n==2:
        def fct(x, h, i, j):
            return polynomial(x, 0, 0, 0, 0, 0, 0, 0, h, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1], parameters[2])
                        
    elif n==1:
        def fct(x, i, j):
            return polynomial(x, 0, 0, 0, 0, 0, 0, 0, 0, i, j)
        parameters, pcov = curve_fit(fct, fieldcut, ampcut)
        fit = fct(field, parameters[0], parameters[1])
            
    ampsubtract = amp - fit
    return (ampsubtract, fit)


## Subtract a baseline from data
### the previous two functions are used

In [ ]:
def baseline (data, Field):
    
    plt.plot(Field, data)
    plt.show()
    
    pos = input('enter the minimum and maximum values of the peaks separated by commas (i.e. min1,max1,min2,max2,...): ')
    
    allpos = []
    for i in np.arange(pos.count(',')):
        a = pos.find(',')
        allpos.append(float(pos[:a]))
        pos = pos[a+1:]
    
    allpos.append(float(pos))
    
    xmin = allpos[::2]
    xmax = allpos[1::2]
    
    happy = 'n'
    
    while happy=='n':
        
        n = float(input('what order polynomial do you want to fit to the data? '))
        ampsubtract = fitpolynomial(n, xmin, xmax, data, field)[0]
        fitpoly = fitpolynomial(n, xmin, xmax, data, field)[1]
        
        fig = plt.figure(figsize=[10,3])
        ax1 = fig.add_subplot(1,2,1)
        ax2 = fig.add_subplot(1,2,2)
        
        ax1.plot(field, data)
        ax1.plot(field, fitpoly)
        
        ax2.plot(field, ampsubtract)
        
        plt.show()
        
        happy = input('are you happy with the subtracted curve? [y/n/end/new]: ')
        
        if happy == 'end':
            break
        
        if happy == 'new':
            #fig, ax1 = plt.subplots()
            
            #ax1.set_xlabel('field')
            #ax1.set_ylabel('amp', color = 'red')
            #ax1.plot(field, amp, color = 'red')
            #ax1.tick_params(axis='y', labelcolor = 'red')
            
            #ax2 = ax1.twinx()
            
            #ax2.set_ylabel('phase', color = 'navy')
            #ax2.plot(field, phase, color = 'navy')
            #ax2.tick_params(axis='y', labelcolor = 'navy')
            
            #fig.tight_layout()
            
            plt.plot(Field, data)
            plt.show()
            
            
            pos = input('enter the minimum and maximum values of the peaks separated by commas (i.e. min1,max1,min2,max2,...): ')
            allpos = []
            for i in np.arange(pos.count(',')):
                a = pos.find(',')
                allpos.append(float(pos[:a]))
                pos = pos[a+1:]
            
            allpos.append(float(pos))
            
            xmin = allpos[::2]
            xmax = allpos[1::2]
            
            happy = 'n'
    
    return ampsubtract


## Normalize a data set to 1
That means that max(data)-min(data) = 1

In [ ]:
def normalize (data):
    a = []
    for i in np.arange(len(data)):
        if np.isnan(data[i]) == True:
            a.append(False)
        else:
            a.append(True)
    b = data[a]
    return data / np.absolute(max(b) - min(b))

## Kramers Kronig Relation
calculates the real part of a susceptibility from a given imaginary part

In [ ]:
def Kramers_Kronig (amp, field):
    amplitude = lambda x: interp1d(field, amp)(x)
    kkr = ft.hilbert(amplitude(field))
    
    return kkr

## Phase corrections

In [ ]:
def phasecorr (Amp, Phase, Field, emin, emax, deltae):
    
    #Amp = Amp / np.absolute(max(Amp) - min(Amp))
    #Phase = Phase / np.absolute(max(Phase) - min(Phase))
    
    
    
    data = [ dict(
        visible = False,
        name = str(epsilon),
        x = Field, 
        y = Amp + epsilon*Phase ) for epsilon in np.arange(emin,emax,deltae) ]
    data[0]['visible'] = True
    
    steps = []
    for i in range(len(data)):
        step = dict(
            method = 'restyle', 
            label = str(np.arange(emin,emax,deltae)[i]),
            args = ['visible', [False] * len(data) ],
            )
        step['args'][1][i] = True # Toggle i'th trace to "visible"
        steps.append(step)
    
    sliders = [ dict(
        active = 10,
        currentvalue = {"prefix": "epsilon: "},
        pad = {"t": 100},
        steps = steps)]
    
    layout = dict( sliders = sliders )
    
    fig = dict( data = data, layout = layout )
    
    py.iplot( fig)
    
    eps = input('enter the value of epsilon you ended up with: ')
    
    amp_p = (Amp+float(eps)*Phase) / np.sqrt(1+float(eps)**2)
    
    phase_kkr = Kramers_Kronig (amp_p, Field)
    
    plot(amp_p, phase_kkr, Field)
    
    return amp_p, phase_kkr, float(eps)


## For ESR data Fitting

In [2]:
# Lorentzian absorption lines - needed for HF-ESR data
def one_Lorentzian (x, x0, A, gamma):
    return -A * gamma / ( (x-x0)**2 + gamma**2 )

def one_Lorentzian_off (x, x0, A, gamma, c):
    return (-A * gamma / ( (x-x0)**2 + gamma**2 )) + c

def two_Lorentzians_off (x, x01, A1, gamma1, x02, A2, gamma2, c):
    return -( A1 * gamma1 / ( (x-x01)**2 + gamma1**2 ) + A2 * gamma2 / ( (x-x02)**2 + gamma2**2 ) ) + c

def three_Lorentzians_off (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, c):
    signal1 = -A1 * gamma1 / ( (x-x01)**2 + gamma1**2 )
    signal2 = -A2 * gamma2 / ( (x-x02)**2 + gamma2**2 )
    signal3 = -A3 * gamma3 / ( (x-x03)**2 + gamma3**2 )
    return signal1 + signal2 + signal3 + c
    

# derivatives of Loentzian absorption lines (including the fit of a background line) - needed for fits of X-ESR data
def one_Lor (x, x0, A, gamma):
    return -2 * A * gamma * (x-x0) / ( (x-x0)**2 + gamma**2 )**2

def one_Lor_w_lin_b (x, x0, A, gamma, d, e):
    return one_Lor(x, x0, A, gamma) + d*x + e

def one_Lor_w_quadr_b (x, x0, A, gamma, c, d, e):
    return one_Lor(x, x0, A, gamma) + c*x**2 + d*x + e

def two_Lor (x, x01, A1, gamma1, x02, A2, gamma2):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2)

def two_Lor_w_lin_b (x, x01, A1, gamma1, x02, A2, gamma2, d, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + d*x + e

def two_Lor_w_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, c, d, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + c*x**2 + d*x + e

def three_Lor_w_lin_b (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, d, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + one_Lor(x, x03, A3, gamma3)+ d*x + e

def three_Lor_w_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, c, d, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + one_Lor(x, x03, A3, gamma3)+ c*x**2 + d*x + e

# px and py have to be defined somewhere before using the function
# they are not supposed to be fit parameters
def two_Lor_w_constrained1_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, c, d):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + c*x**2 + d*x + (py - c*px**2 - d*px)

def two_Lor_w_constrained2_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, c, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + c*x**2 + ( (py-e)/px -c*px )*x + e

def two_Lor_w_constrained3_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, d, e):
    return one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + (py-e-d*px)/px/px*x**2 + d*x + e

def three_Lor_w_constrained1_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, c, d):
    a = one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + one_Lor(x, x03, A3, gamma3)+ c*x**2 + d*x + (py - c*px**2 - d*px)
    return a

def three_Lor_w_constrained2_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, c, e):
    a = one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + one_Lor(x, x03, A3, gamma3)+ c*x**2 + ( (py-e)/px -c*px )*x + e
    return a

def three_Lor_w_constrained3_quadr_b (x, x01, A1, gamma1, x02, A2, gamma2, x03, A3, gamma3, d, e):
    a = one_Lor(x, x01, A1, gamma1) + one_Lor(x, x02, A2, gamma2) + one_Lor(x, x03, A3, gamma3)+ (py-e-d*px)/px/px*x**2 + d*x + e
    return a


# include phase mixing into the fit funciton
def one_Lor_w_phase (B, B0, A, gamma, mixing_angle):
    signal = one_Lorentzian(B, B0, A, gamma) + np.sin(mixing_angle*np.pi/180) * (A * (B-B0) / ( (B-B0)**2 + gamma**2 ))
    return signal / np.sqrt( 1 + (np.sin(mixing_angle*np.pi/180))**2 )

def one_Lor_w_phase_and_offset (B, B0, A, gamma, mixing_angle, offset):
    signal = one_Lorentzian(B, B0, A, gamma) + np.sin(mixing_angle*np.pi/180) * (A * (B-B0) / ( (B-B0)**2 + gamma**2 ))
    return signal / np.sqrt( 1 + (np.sin(mixing_angle*np.pi/180))**2 ) + offset

def one_Lor_derivative_w_phase (B, B0, A, gamma, mixing_angle):
    #amp_derivative = one_Lor(B, B0, A, gamma) 
    #phase_derivative = A * ( 1 / ( (B-B0)**2 + gamma**2 ) - 2*(B-B0)**2 / ( (B-B0)**2 + gamma**2 )**2 )
    #signal = amp_derivative + np.sin(mixing_angle*np.pi/180) * phase_derivative
    signal = -np.gradient(one_Lor_w_phase (B, B0, A, gamma, mixing_angle), B)
    return signal

def one_Lor_derivative_w_phase_quadr_b (B, B0, A, gamma, mixing_angle, c, d, e):
    signal = one_Lor_derivative_w_phase (B, B0, A, gamma, mixing_angle) + c*B**2 + d*B + e
    return signal





def g_factor (frequency,B_resonance):
    g = frequency*10000/B_resonance[:,0]/13.98
    g_err = B_resonance[:,1]*frequency*10000/B_resonance[:,0]/B_resonance[:,0]/13.98
    return g, g_err

def g_eff (frequency,Bres, Berr):
    g = frequency*10000/Bres/13.98
    g_err = Berr*frequency*10000/Bres/Bres/13.98
    return g, g_err

# switch to elements i, j of a list
def switch (array, i, j):
    a = array[i]
    b = array[j]
    array[i] = b
    array[j] = a
    return array


# in the Li case
# change of attenuation at T[13]-T[14] and T[34]-T[35]
def rescale_A_Li (A, dA):
    A_switch = A #switch(A, 34, 35)
    Aerr_switch = dA #switch(dA, 34, 35)
        
    A_new = np.zeros(len(A))
    
    A_new[:14] = A_switch[:14]
    A_new[14:35] = A_switch[14:35] * A_switch[13] / A_switch[14]
    A_new[35:] = A_switch[35:] * A_switch[13] / A_switch[14] * A_switch[34] / A_switch[35]
    
    Aerr_new = np.zeros(len(dA))
    Aerr_new[:14] = Aerr_switch[:14]
    Aerr_new[14:35] = Aerr_switch[14:35] * A_switch[13] / A_switch[14]
    Aerr_new[35:] = Aerr_switch[35:] * A_switch[13] / A_switch[14] * A_switch[34] / A_switch[35]
    
    #A_new[35:] = A_switch[35:]
    #A_new[14:35] = A_switch[14:35] * A_switch[35] / A_switch[34]
    #A_new[:14] = A_switch[:14] * A_switch[14] / A_switch[13] * A_switch[35] / A_switch[34]
    
    #Aerr_new = np.zeros(len(dA))
    #Aerr_new[35:] = Aerr_switch[35:]
    #Aerr_new[14:35] = Aerr_switch[14:35] * A_switch[35] / A_switch[34]
    #Aerr_new[:14] = Aerr_switch[:14] * A_switch[14] / A_switch[13] * A_switch[35] / A_switch[34]

    
    return A_new, Aerr_new


# in the Na case
# change of attenuation at T[12]-T[13]
def rescale_A_Na (A, dA):
    A_new = np.zeros(len(A))
    
#    A_new[:13] = A[:13,0] 
#    A_new[13:] = A[13:,0] * A[12,0]/A[13,0]   
    
    A_new[:13] = A[:13] * A[13]/A[12]
    A_new[13:] = A[13:]    
    
    Aerr_new = np.zeros(len(A))
    Aerr_new[:13] = dA[:13] * A[13]/A[12]
    Aerr_new[13:] = dA[13:] 
    
    
    #A_new[:13] = A[:13]
    #A_new[13:] = A[13:] * A[12]/A[13]
    
    #Aerr_new[:13] = dA[:13] 
    #Aerr_new[13:] = dA[13:] * A[12]/A[13]
    
    
    
    
    return A_new, Aerr_new


def rescale_A (A, dA, compound):
    if compound == 'Li':
        out = rescale_A_Li(A, dA)
    elif compound == 'Na':
        out = rescale_A_Na(A, dA)
    else:
        raise Exception('The only two possible input arguments for "compound" are "Li" or "Na"')
    return out

# Plot several ESR-data sets with the fit curves
## this is a general plot function allowing for various final plots
## but only for fits with several Lorentzians

In [1]:
# temp can either be a single number or a list of numbers
# compound can be 'Li' or 'Na' (has to be a string!)
# ncolumns gives the number of columns if there should be several different temperatures
# grid: 'yes' if you want to show grid lines, anything else if you don't
# raw: 'yes' if you want to show the actual raw data (even without background subtracted); anything else if you don't
# additional_folder: if the individual_fit_curves files are not directly in "...\\individual_fit_curves" here you
# have to enter the additional folder name (if they are directly here enter 'no')
# one_Lor: do you also want to show the fit of one Lorentzian: 'yes' or 'no'
def plot_fits (temp, compound, ncolumns, grid, raw, additional_folder, one_Lor):
    
    
    folder_general = "C:\\Users\\F25_1.307_b\\Box Sync\\Klingeler_Masterarbeit\\howardevansite\\"
    
    if compound == 'Li':
        folder_specific = 'LiCuFe2(VO4)3 powder\\X-band\\21_02_2019'
    
    elif compound == 'Na':
        folder_specific = 'NaCuFe2(VO4)3 powder\\X-band\\19_02_2019'
        
    else:
        raise Exception('The variable "compound" can only take Li or Na (and they have to be strings!). You entered: {}'.format(compound))
        

    # list of the names of all the files in "folder" 
    if additional_folder == 'no':
        folder1 = folder_general + folder_specific + '\\individual_fit_curves'
    else:
        folder1 = folder_general + folder_specific + '\\individual_fit_curves\\' + additional_folder
    files = [i for i in os.listdir(folder1)]
    files = np.array(os.listdir(folder1))
    files = files[ [i.find('individual')!=-1 for i in files] ]
    
    # write function that finds the temperature in the file names
    def find_T (filename):
        lower = filename.find('s')
        upper = filename.find('K')
        T = filename[lower+2:upper-1]
        return float(T)
    
    
    # create an array of temperatures of the filenames
    file_T = np.array([find_T(i) for i in files])
    
    
###################### gives an array of all the temperature that are the closest to  the given temperature
    # only gives more than one entry if several temperatures have the same distance from the given one
    if type(temp)==float or type(temp)==int or len(temp)==1:
        possible_temperatures = file_T[abs(file_T-temp) - min(abs(file_T-temp)) < 0.01]
    
    elif type(temp) == list:
        mask_lower = file_T >= temp[0]
        mask_upper = file_T <= temp[1]
        mask = [ np.all([mask_lower[i], mask_upper[i]]) for i in np.arange(len(mask_upper)) ]
        possible_temperatures = file_T[mask]
    
    else:
        raise Exception('The temperature you entered does not have the right type. Only enter a single value or a list of integers or floats.')
        
        
    if len(possible_temperatures) == 0:
        raise Exception('There is no measured data file in the chosen temperature range. Choose a wider range.')
        
           
        
    
    
###################### create array of filenames of data taken at the temperatures in 'possible_temperatures'
    # get the files with the temperatures from possible_temperatures
    mask = []
    for t in possible_temperatures:
        mask.append( abs(file_T-t) < 0.01 )
    mask = np.array(mask)
    
    final_mask = []
    for i in np.arange(len(file_T)):
        final_mask.append(np.any(np.transpose(mask)[i]))
    
    files = files[final_mask]
    
    # since you're already at it also import the right filename from fits of only one Lorentzian
    if one_Lor == 'yes':
        folder = folder_general + folder_specific + '\\individual_fit_curves'
        files0 = np.array(os.listdir(folder+'\\single_Lorentzian'))
        files0 = files0[final_mask]
    
#######################################################################################
#######################################################################################
# plot stuff
    
    files_T = [find_T(i) for i in files]
    files = files[np.argsort(files_T)]
    files_T = sorted(files_T)
    
    if len(files) < ncolumns:
        ncolumns = len(files)
    else:
        ncolumns = ncolumns
    nrows = math.ceil( len(files) / ncolumns )
    
    
    fig, axes = plt.subplots( nrows, ncolumns, sharex=True, sharey=True, figsize=(15*ncolumns,10*nrows) )
    
    if nrows > 1:
        
        for i in np.arange(len(files)+1)[1:]:
            
            d = data_import(folder1 +'\\'+files[i-1], 3)
                        
            if len(d) < 8:
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[2])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[3])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[4])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[6])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].scatter(d[0], d[1]+d[6], s=1, c = 'pink')
                if one_Lor == 'yes':
                    d0 = data_import(folder +'\\single_Lorentzian\\'+files0[i-1], 3)
                    axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d0[0], d0[2])
                
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].legend(['sum', 'L1', 'L2', 'background',
                                                                     'data minus background', 'raw data'], fontsize = 20)
            
            else:
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[2])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[3])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[4])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[5])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d[0], d[7])
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].scatter(d[0], d[1]+d[7], s=1, c = 'pink')
                if one_Lor == 'yes':
                    axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].plot(d0[0], d0[2])
                    
                axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].legend(['sum', 'L1', 'L2', 'L3', 'background',
                                                                     'data minus background', 'raw data'], fontsize = 20)
            
            #axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].text(0.6,15, str(files_T[i-1])+ ' K', color='black', fontsize=30)
            axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].set_title(str(files_T[i-1])+' K', fontsize = 30)
            if grid == 'yes':
                axes.grid(color='gray', linestyle='--', linewidth=1)
           
        
    elif ncolumns > 1:
        
        for i in np.arange(len(files)+1)[1:]:
            
            d = data_import(folder1 +'\\'+files[i-1], 3)
                        
            if len(d) < 8:
                axes[i-1].scatter(d[0], d[1], s=5, c = 'gray')
                axes[i-1].plot(d[0], d[2])
                axes[i-1].plot(d[0], d[3])
                axes[i-1].plot(d[0], d[4])
                axes[i-1].plot(d[0], d[6])
                axes[i-1].scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes[i-1].scatter(d[0], d[1]+d[6], s=1, c = 'pink')
                if one_Lor == 'yes':
                    d0 = data_import(folder +'\\single_Lorentzian\\'+files0[i-1], 3)
                    axes[i-1].plot(d0[0], d0[2])
                axes[i-1].legend(['sum', 'L1', 'L2', 'background', 'data minus background', 'raw data'], fontsize = 20)
                
            else:
                axes[i-1].plot(d[0], d[2])
                axes[i-1].plot(d[0], d[3])
                axes[i-1].plot(d[0], d[4])
                axes[i-1].plot(d[0], d[5])
                axes[i-1].plot(d[0], d[7])
                axes[i-1].scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes[i-1].scatter(d[0], d[1]+d[7], s=1, c = 'pink')
                if one_Lor == 'yes':
                    axes[i-1].plot(d0[0], d0[2])
                axes[i-1].legend(['sum', 'L1', 'L2', 'L3', 'background', 'data minus background', 'raw data'], fontsize = 20)
            
            #axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].text(0.6,15, str(files_T[i-1])+ ' K', color='black', fontsize=30)
            axes[i-1].set_title(str(files_T[i-1])+' K', fontsize = 30)
            if grid == 'yes':
                axes.grid(color='gray', linestyle='--', linewidth=1)
            
            
    else:
        
        for i in np.arange(len(files)+1)[1:]:
            
            d = data_import(folder1 +'\\'+files[i-1], 3)
                        
            if len(d) < 8:
                axes.plot(d[0], d[2])
                axes.plot(d[0], d[3])
                axes.plot(d[0], d[4])
                axes.plot(d[0], d[6])
                axes.scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes.scatter(d[0], d[1]+d[6], s=1, c = 'pink')
                if one_Lor == 'yes':
                    d0 = data_import(folder +'\\single_Lorentzian\\'+files0[i-1], 3)
                    axes.plot(d0[0], d0[2])
                axes.legend(['sum', 'L1', 'L2', 'background', 'data minus background', 'raw data'], fontsize = 20)
                                
                
            else:
                axes.plot(d[0], d[2])
                axes.plot(d[0], d[3])
                axes.plot(d[0], d[4])
                axes.plot(d[0], d[5])
                axes.plot(d[0], d[7])
                axes.scatter(d[0], d[1], s=5, c = 'gray')
                if raw == 'yes':
                    axes.scatter(d[0], d[1]+d[7], s=1, c = 'pink')
                if one_Lor == 'yes':
                    axes.plot(d0[0], d0[2])
                axes.legend(['sum', 'L1', 'L2', 'L3', 'background', 'data minus background', 'raw data'], fontsize = 20)
                
            #axes[math.ceil(i/ncolumns)-1, (i-1)%ncolumns].text(0.6,15, str(files_T[i-1])+ ' K', color='black', fontsize=30)
            axes.set_title(str(files_T[i-1])+' K', fontsize = 30)
            if grid == 'yes':
                axes.grid(color='gray', linestyle='--', linewidth=1)
            
    
    plt.subplots_adjust(wspace=0.01, hspace=0.3)
    
    b=0
    for i in np.arange(nrows):
        for j in np.arange(ncolumns):
            if b > len(files)-1:
                axes[i,j].axis('off')
                b=b+1
            else:
                b=b+1
    
    
    plt.show()